In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
sns.set_style("white")

In [ ]:
# Import required packages
import spacy
from spacy import displacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [ ]:
import string


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV,LinearRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

# Load data:
For the first part we import the data to get the firts informations about the data.  All the features in the dataset are:
1-id
2-keyword
3-location
4-text
5-target

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/michalis0/DataMining_and_MachineLearning/master/project/training_data.csv")
df.head(3)

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1


In [ ]:
df.shape

(6471, 5)

# Target values are composed of 0 and 1. Whether these twits are about a real disaster or not is the focus of the concern.


In [ ]:
print(df.target.unique())

array([0, 1])

In [ ]:
#baserate
max(len(df[df["target"] == 0]) / len(df), len(df[df["target"] == 1]) / len(df))

0.5719363313243703

##LOCATION-TARGET
Is there a meaningful relation in the  the country level data. i.e: Can we expect that in certain places people are more prone to diffuse fake disaster news:

In [ ]:
df2=df[['location','target']].dropna()
print(df2.shape)
X=df['location'].dropna()


(4330, 2)


In [ ]:
df['location'].unique().shape

(2922,)

In [ ]:
#Categorizing the location 
X.unique().shape # around 3000 unique locations. that would not be a good fature to classify the 

(2921,)

###KEYWORD-TARGET
We can try to analyse the predictability of the keywords on the target. in another words, if for sertain types of disasters, whether people are more accurate, and for certain types people are more prone to hoax.

In [ ]:
df3=df[['keyword','target']].dropna()
print("sjhape of df2:", df2.shape)
X=df3['keyword']
y=df3['target']
#How many unique keyword : 221
print(" unique cat in keyword:", X.unique().shape)



sjhape of df2: (4330, 2)
 unique cat in keyword: (221,)


In [ ]:
#a label encoding could be used to encode the keywords.
myLabEn = LabelEncoder()
df3_encoded = df3.apply(myLabEn.fit_transform) #
#df3_encoded


In [ ]:
#Data prepatation
X=df3_encoded[['keyword']]
y=df3.target

#Splitting train and Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)

X_train

,keyword
1741,111
4259,78
4237,81
3893,7
2433,155
...,...
973,88
1679,120
3346,173
1700,42


In [ ]:
#Fitting
LR=LogisticRegression(solver='lbfgs',  max_iter=1000,random_state=3)
LR.fit(X_train, y_train)

# Accuracy of training set and test set
LR.score(X_train, y_train)

LR.score(X_test, y_test)
print("Accuracy of train and test sets:",LR.score(X_train, y_train), LR.score(X_test, y_test))

Accuracy of train and test sets: 0.5693686671862821 0.5926791277258567


In [ ]:
LRcv=LogisticRegressionCV(solver='lbfgs', cv=5,  max_iter=1000,random_state=3)
LRcv.fit(X_train, y_train)

# Accuracy of training set and test set
LRcv.score(X_train, y_train)

LRcv.score(X_test, y_test)
print("Accuracy of train and test sets:",LRcv.score(X_train, y_train), LRcv.score(X_test, y_test))

#It seems the single relationsshipe between keyword and the target is not that meaningful. the accuracy rate is almost near to the base rate. 

Accuracy of train and test sets: 0.5693686671862821 0.5926791277258567


In [ ]:
# Predictions
y_pred = LRcv.predict(X_test)
#Confusion matrix
confusion_matrix(y_test, y_pred)
y_pred=pd.DataFrame(y_pred)
y_pred

,0
0,0
1,0
2,0
3,0
4,0
...,...
1279,0
1280,0
1281,0
1282,0


In [ ]:
!git init

Initialized empty Git repository in /content/.git/


TF-IDF feature vectors
create the TF-IDF feature vectors for the processed tweetes. These will construct you data features that you will use to train a classifier.